In [8]:
import os, re
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
from tqdm import tqdm_notebook as tqdm
import random
from lxml.html import fromstring
import time
import pandas as pd

ua = UserAgent(verify_ssl=False)

In [5]:
def get_proxies():
    #url = 'https://free-proxy-list.net/'
    url = 'https://free-proxy-list.net/anonymous-proxy.html'
    response = requests.get(url)
    parser = fromstring(response.text)
    proxies = set()
    for i in parser.xpath('//tbody/tr')[:500]:
        if i.xpath('.//td[7][contains(text(),"yes")]'):
            #Grabbing IP and corresponding PORT
            proxy = ":".join([i.xpath('.//td[1]/text()')[0], i.xpath('.//td[2]/text()')[0]])
            proxies.add(proxy)
    return proxies
proxies = list(get_proxies())
len(proxies)

76

In [3]:
proxies

['185.39.197.101:8080',
 '103.232.33.6:32231',
 '43.229.92.44:23500',
 '103.73.224.196:8080',
 '109.72.103.40:61103',
 '92.46.35.166:41578',
 '103.197.48.90:23500',
 '95.67.65.18:53281',
 '103.70.146.226:8080',
 '41.215.244.66:38460',
 '24.217.192.238:47397',
 '35.199.96.12:80',
 '94.154.85.214:8080',
 '212.49.84.113:65103',
 '79.106.108.110:32231',
 '119.82.253.182:44145',
 '85.11.151.92:60903',
 '176.9.221.148:3128',
 '186.232.48.70:53281',
 '178.158.218.230:33347',
 '176.221.115.26:32752',
 '46.55.219.131:38812',
 '46.101.142.157:8080']

# Выкачка адресов

In [6]:
def get_adresses(i, proxies):
    headers = {'User-Agent': ua.random}
    x = random.choice(proxies)
    proxy = {'https':x, 'http': x}
    session = requests.session()
    r = session.get("http://www.httpbin.org/ip", proxies=proxy, headers=headers)
    print (i+1, r.json()['origin'])
    r = session.get('https://www.avito.ru/moskva/vakansii?p={}&view=list'.format(i+1), headers=headers, proxies=proxy)
    html = r.text
    soup = BeautifulSoup(html, 'lxml')
    links = soup.find_all('a', attrs={'class':'description-title-link'})
    links = ['https://www.avito.ru'+item['href'] for item in links]
    return links

In [7]:
def get_all_adresses():
    with open('url_list.txt','w',encoding='utf-8') as f:
        for i in tqdm(range(100)):
            proxies = list(get_proxies())
            print (len(proxies), end='\t')
            try:
                links = get_adresses(i, proxies)
            except:
                links = get_adresses(i, proxies)
            for url in links:
                f.write(url+'\n')

In [8]:
def get_adresses(i):
    headers = {'User-Agent': ua.random}
    session = requests.session()
    r = session.get('https://www.avito.ru/moskva/vakansii?p={}&view=list'.format(i+1), headers=headers)
    html = r.text
    soup = BeautifulSoup(html, 'lxml')
    links = soup.find_all('a', attrs={'class':'description-title-link'})
    links = ['https://www.avito.ru'+item['href'] for item in links]
    return links

def get_all_adresses():
    with open('url_list.txt','w',encoding='utf-8') as f:
        for i in tqdm(range(100)):
            t = True
            while t:
                try:
                    links = get_adresses(i)
                    for url in links:
                        f.write(url+'\n')
                    x = random.uniform(0.5,2.9)
                    time.sleep(x)
                    #print (i+1, x)
                    t = False
                except:
                    pass

In [3]:
get_all_adresses() #03:45

In [4]:
os.makedirs('corpora')

In [2]:
def load_links():
    with open('url_list.txt','r') as f:
        links = f.read().split('\n')
    return links

In [11]:
def get_data_from_url(url):
    d = {}
    headers = {'User-Agent': ua.random}
    session = requests.session()
    r = session.get(url, headers=headers)
    html = r.text
    #print (html)
    soup = BeautifulSoup(html, 'lxml')
    x = soup.find_all('span', attrs={'class':'title-info-title-text'})
    if x: d['title'] = x[0].text.strip()
    else: d['title'] = None
    x = soup.find_all('div', attrs={'class':'item-description-text'})
    if x: d['description'] =  re.sub('<.*?>','\n', str(x[0])).strip()
    else: d['description'] = ''
    for i in soup.find_all('li', attrs={'class':'item-params-list-item'}):
        kv = str(i.text).strip().split(':')
        d[kv[0].strip()] = kv[1].strip()
    address = soup.find_all('span', attrs={'class':'item-map-address'})
    if address: d['address'] = address[0].text.strip()
    return d

In [11]:
get_data_from_url('https://www.avito.ru/moskva/vakansii/pedagog_po_angliyskomu_yazyku_1176420097')

{'title': 'Педагог по английскому языку',
 'description': 'В Семейный клуб «Вдохновение» требуется педагог по английскому языку для проведения групповых занятий в вечернее время с детьми дошкольного и младшего школьного возраста. \nТребования: \n- высшее образование;\n- опыт работы с детьми;\n- доброжелательность, любовь к детям, стремление к творческому и профессиональному росту;\n- медицинская книжка.\nОбязанности:\n- проведение занятий только на английском языке (без перевода на русскую речь);\n- подготовка к занятиям, приветствуется инициатива.\nУсловия:\n- оформление по ТК РФ;\n- частичная занятость; \n- 2-4 раза в неделю;\n- в группе 6-8 детей;\n- оплата сдельная.\n\nСобеседование состоится после рассмотрения письменных резюме. Срок подачи резюме - до 05 октября 2018 года. Резюме принимаются по e-mail. Наличие фото к резюме обязательно!',
 'Сфера деятельности': 'Образование, наука',
 'График работы': 'свободный график',
 'Опыт работы': 'более 1 года',
 'address': 'м.\xa0Планерная

In [15]:
def get_all_data():
    data = pd.DataFrame()
    links = load_links()
    random.shuffle(links)
    k = 0
    for url in tqdm(links):
        try:
        #if True:
            d = get_data_from_url(url)
            d['name'] = url.split('/')[-1]
            if d:
                text = d['description']
                if 'description' in d: del d['description']
                data = data.append(d, ignore_index=True)
                with open('./corpora/'+d['name'], 'w', encoding='utf-8') as f:
                    f.write(text)   
            x = random.uniform(0.8,3.7)
            time.sleep(x)
            data.to_csv('data.csv', sep='\t')
        except:
            k += 1
            print ('|', end='\t')
    print ('\n', k, k/len(links))

In [16]:
get_all_data() #7:50:39, 1 ошибка

|	
 1 9.270418095856123e-05


## Метаинформация

In [24]:
data = pd.read_csv('data.csv', sep='\t', index_col=0)

In [25]:
data = data.drop_duplicates()

In [29]:
data.head()

,address,name,title,График работы,Опыт работы,Сфера деятельности
0,"Москва, м. Бульвар Дмитрия Донского, Старокача...",menedzher_-konsultant_951375511,Менеджер -консультант,полный день,не имеет значения,Продажи
1,NaN,ohrannik_v_tts_vodnyy_1201120983,Охранник в ТЦ Водный,сменный график,не имеет значения,"Охрана, безопасность"
2,"Москва, м. Дмитровская, Складочная улица, 1с13",uborka_ofisa_1535259521,Уборка офиса,NaN,NaN,NaN
3,"м. Охотный ряд, \n Москва",naborschik_zakazov_1502732417,Наборщик заказов,сменный график,более 1 года,"Транспорт, логистика"
4,"Москва, м. Нахимовский проспект, Нахимовский п...",voditel_taksi_1394522421,Водитель такси,сменный график,более 3 лет,Автомобильный бизнес


In [30]:
data.shape

(9947, 6)

In [28]:
data.to_csv('data.csv', sep='\t')